In [1]:
# If you get a SciPy error when installing Emukit, build it from source:

# git clone https://github.com/amzn/Emukit.git
# cd Emukit
# pip install -r requirements/requirements.txt
# python setup.py develop

In [6]:
import numpy as np
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs import RandomDesign
from emukit.core.loop import UserFunctionWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

from sumo_grid_simulation import grid_simulation

### Pseudocode

(taken from L48 lectures) Fitting a gaussian process to a simulator using Emukit takes the following form:

```
initialize GP with some randomly chosen points
while stopping condition is not met:
    compute candidate point(s) using GP and acquisition funciton (expected improvement) -> new point(s)
    evaluate this new point with our simulator/user function -> observation
    update model with new observation -> new GP
```

### User Function / Simulator
 
This is the function we want to understand. Namedly, how do parameters such as speed limit, traffic light timing, junction type and number of lanes per road, affect (1) the total carbon emissions generated by traffic in a road network, and (2) the average time it takes for vehicles to move through this network. 

*A complete list of parmeters analysed is discussed in our report.*

outputs returned by Simulator.simulate():
```
CO
CO2
HC
NOx
PMx
fuel
noise
num_emissions_samples
departDelay
departDelayWaiting
duration
routeLength
speed
timeLoss
waitingTime```

In [12]:
def dummy_user_function(X):
    speed = X[:,0]
    num_lanes = X[:,1]
    traffic_light_time = X[:,2]
    
    CO = np.sin(speed)
    CO2 = np.cos(num_lanes)
    HC = np.tanh(traffic_light_time)
    
    return np.stack((CO, CO2, HC)).T


simulator = grid_simulation.Simulator()

def user_function(X):  # emukit doesnt pass named args, just an NxM ndarray
    
    result = []
    
    for speed_limit, num_lanes in X:
        s = simulator.simulate(
            gridSize = 4,
            edgeMaxSpeed = int(speed_limit),  # casting to int is essential
            numberOfLanes = int(num_lanes)
        )
        N = s['num_emissions_samples']  # we want global average emissions
        # todo parameterise
#         result.append([s['CO']/N, s['CO2']/N, s['NOx']/N, s['waitingTime']])
        result.append(s['waitingTime'])
    
    return np.expand_dims(np.array(result), 1)  # expand dims is essential or the acquition function breaks

### Model (GP)

Our surrogate model is our emulator. In this case, a gaussian process. Since we have both continuous and categorical inputs we convert categorical variables to their one-hot encoded counterparts. We want to predict multiple continuous outputs - fortunately GPy handles this for us.

#### Model Inputs

In [13]:
speed = ContinuousParameter('speed', min_value=1, max_value=70)
lanes = DiscreteParameter('lanes', domain=[1,2,3])

parameter_space = ParameterSpace([speed, lanes])

#### Initialize Model / Emulator (GP)

In [14]:
design = RandomDesign(parameter_space)  # initialize with random points
num_data_points = 5
init_X = design.get_samples(num_data_points)
init_Y = user_function(init_X)

a
a
a
a
b
b
b
b
c
c
c
c
d
d
d
d
Success.

calling /usr/local/opt/sumo/share/sumo/bin/duarouter -n /Users/Matteo/PycharmProjects/TrafficEmu/grid_simulation/grid.net.xml -r /Users/Matteo/PycharmProjects/TrafficEmu/grid_simulation/veh_passenger.trips.xml --ignore-errors --begin 0.0 --end 3600.0 --no-step-log --additional-files /Users/Matteo/PycharmProjects/TrafficEmu/vehicle_generator/veh.add.xml --no-warnings -o /Users/Matteo/PycharmProjects/TrafficEmu/grid_simulation/veh_passenger.rou.xml
Success.
calling /usr/local/opt/sumo/share/sumo/bin/duarouter -n /Users/Matteo/PycharmProjects/TrafficEmu/grid_simulation/grid.net.xml -r /Users/Matteo/PycharmProjects/TrafficEmu/grid_simulation/veh_passenger.trips.xml --ignore-errors --begin 0.0 --end 3600.0 --no-step-log --additional-files /Users/Matteo/PycharmProjects/TrafficEmu/vehicle_generator/veh.add.xml --no-warnings -o /Users/Matteo/PycharmProjects/TrafficEmu/grid_simulation/veh_passenger.trips.xml.tmp --write-trips
Success.

 Retrying in 1 se

In [15]:
init_X.shape, init_Y.shape

((5, 2), (5, 1))

In [16]:
init_X, init_Y

(array([[27.83660147,  1.        ],
        [15.25169918,  2.        ],
        [39.61495862,  3.        ],
        [37.70493599,  2.        ],
        [ 8.71245273,  2.        ]]),
 array([[0.11],
        [0.1 ],
        [0.16],
        [0.14],
        [0.12]]))

In [17]:
emulator = GPy.models.GPRegression(init_X, init_Y)
emukit_model = GPyModelWrapper(emulator)
emulator

GP_regression.,value,constraints,priors
rbf.variance,1.0,+ve,
rbf.lengthscale,1.0,+ve,
Gaussian_noise.variance,1.0,+ve,


### Optimization

#### Acquisition Function

In [18]:
expected_improvement = ExpectedImprovement(model=emukit_model)

#### Bayesian Optimization

In [20]:
bayes_loop = BayesianOptimizationLoop(
    model = emukit_model,
    space = parameter_space,
    acquisition = expected_improvement,
    batch_size = 1
)

In [21]:
bayes_loop.run_loop(user_function, 5)

Optimization restart 1/1, f = -9.441204971138246
a
a
a
a
b
b
b
b
c
c
c
c
d
d
d
d
Success.


calling /usr/local/opt/sumo/share/sumo/bin/duarouter -n /Users/Matteo/PycharmProjects/TrafficEmu/grid_simulation/grid.net.xml -r /Users/Matteo/PycharmProjects/TrafficEmu/grid_simulation/veh_passenger.trips.xml --ignore-errors --begin 0.0 --end 3600.0 --no-step-log --additional-files /Users/Matteo/PycharmProjects/TrafficEmu/vehicle_generator/veh.add.xml --no-warnings -o /Users/Matteo/PycharmProjects/TrafficEmu/grid_simulation/veh_passenger.rou.xml
Success.
calling /usr/local/opt/sumo/share/sumo/bin/duarouter -n /Users/Matteo/PycharmProjects/TrafficEmu/grid_simulation/grid.net.xml -r /Users/Matteo/PycharmProjects/TrafficEmu/grid_simulation/veh_passenger.trips.xml --ignore-errors --begin 0.0 --end 3600.0 --no-step-log --additional-files /Users/Matteo/PycharmProjects/TrafficEmu/vehicle_generator/veh.add.xml --no-warnings -o /Users/Matteo/PycharmProjects/TrafficEmu/grid_simulation/veh_passenger.trips

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/1, f = 28.273464713981156
a
a
a
a
b
b
b
b
c
c
c
c
d
d
d
d
Success.


calling /usr/local/opt/sumo/share/sumo/bin/duarouter -n /Users/Matteo/PycharmProjects/TrafficEmu/grid_simulation/grid.net.xml -r /Users/Matteo/PycharmProjects/TrafficEmu/grid_simulation/veh_passenger.trips.xml --ignore-errors --begin 0.0 --end 3600.0 --no-step-log --additional-files /Users/Matteo/PycharmProjects/TrafficEmu/vehicle_generator/veh.add.xml --no-warnings -o /Users/Matteo/PycharmProjects/TrafficEmu/grid_simulation/veh_passenger.rou.xml
Success.
calling /usr/local/opt/sumo/share/sumo/bin/duarouter -n /Users/Matteo/PycharmProjects/TrafficEmu/grid_simulation/grid.net.xml -r /Users/Matteo/PycharmProjects/TrafficEmu/grid_simulation/veh_passenger.trips.xml --ignore-errors --begin 0.0 --end 3600.0 --no-step-log --additional-files /Users/Matteo/PycharmProjects/TrafficEmu/vehicle_generator/veh.add.xml --no-warnings -o /Users/Matteo/PycharmProjects/TrafficEmu/grid_simulation/veh_passenger.trips

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


### Evaluate

In [22]:
init_X, init_Y

(array([[27.83660147,  1.        ],
        [15.25169918,  2.        ],
        [39.61495862,  3.        ],
        [37.70493599,  2.        ],
        [ 8.71245273,  2.        ]]),
 array([[0.11],
        [0.1 ],
        [0.16],
        [0.14],
        [0.12]]))

In [23]:
design = RandomDesign(parameter_space)  # initialize with random points
num_data_points = 10
test_X = design.get_samples(num_data_points)

test_Y, test_Y_variance = emukit_model.predict(test_X)

In [24]:
test_Y  # looks reasonable...

array([[0.24023451],
       [0.24023452],
       [0.24023457],
       [0.24023461],
       [0.24023461],
       [0.24023458],
       [0.24023462],
       [0.2402346 ],
       [0.24023451],
       [0.24023454]])

### Scratch

In [25]:
predicted_y = []
predicted_std = []

real_x = np.arange(0, 70, 0.2)
real_y = np.sin(real_x)

for x in real_x:
    y, var = emukit_model.predict(np.array([[x]]))
    std = np.sqrt(var)
    predicted_y.append(y)
    predicted_std.append(std)

predicted_y = np.array(predicted_y).flatten()
predicted_std = np.array(predicted_std).flatten()
    
plt.title('Learning function sin(x) with Emukit')
plt.xlabel('x')
plt.ylabel('y', rotation=None)
plt.plot(real_x, real_y, c='r', )
plt.plot(real_x, predicted_y)
plt.legend(['True function', 'Estimated function'], loc='lower right')
plt.fill_between(real_x, predicted_y - 2 * predicted_std, predicted_y + 2 * predicted_std, alpha=.5);

IndexError: index 1 is out of bounds for axis 1 with size 1

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

real_x = np.arange(x_min, x_max, 0.2)
real_y = np.sin(real_x)

plt.title('Learning function sin(x) with Emukit')
plt.xlabel('x')
plt.ylabel('y', rotation=None)
plt.plot(real_x, real_y, c='r')
plt.scatter(loop.loop_state.X[:, 0].tolist(), loop.loop_state.Y[:, 0].tolist());
plt.legend(['True function', 'Acquired datapoints'], loc='lower right');

In [ ]:
EdgeType is the same as numLanes